CELL 1

In [ ]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')

if not os.path.exists("Depth-Anything-3"):
    !git clone https://github.com/ByteDance-Seed/Depth-Anything-3
    %cd Depth-Anything-3
    !pip install -r requirements.txt
    !pip install pycolmap triton
    !pip install -e .
    %cd /content
else:
    print("Depth-Anything-3 đã tồn tại")

from google.colab import output
output.enable_custom_widget_manager()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Depth-Anything-3 đã tồn tại.


CELL 2

DA3-LARGE

In [ ]:
import torch
import numpy as np
import os
import cv2
import zipfile
import gc
from pathlib import Path
from depth_anything_3.api import DepthAnything3

ZIP_PATH = "/content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip"
OUTPUT_DIR = "/content/drive/MyDrive/3D_reconstruction/simcol_output_blind_pose"
OUTPUT_DEPTH_DIR = os.path.join(OUTPUT_DIR, "depth_pose_data")

MODEL_NAME = "da3-large"
BATCH_SIZE = 18

FX, FY = 227.60416, 237.5
CX, CY = 227.60416, 237.5

os.makedirs(OUTPUT_DEPTH_DIR, exist_ok=True)

# Giải nén
EXTRACT_PATH = "/content/dataset_simcol_rgb"
if not os.path.exists(EXTRACT_PATH):
    print(f"Đang giải nén dataset...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)

# List
image_extensions = {'.png', '.jpg'}
all_image_paths = sorted([
    str(p) for p in Path(EXTRACT_PATH).rglob("*")
    if p.suffix.lower() in image_extensions
])
total_imgs = len(all_image_paths)
print(f"Tổng số ảnh: {total_imgs}")

# Intrinsic
K = np.array([[FX, 0, CX], [0, FY, CY], [0, 0, 1]], dtype=np.float32)

# LOAD MODEL
print(f"load model {MODEL_NAME}...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DepthAnything3.from_pretrained(f"depth-anything/{MODEL_NAME}").to(device)

# BATCH
print(f"🏃 Running Batch")

for i in range(0, total_imgs, BATCH_SIZE):
    batch_end = min(i + BATCH_SIZE, total_imgs)
    batch_paths = all_image_paths[i : batch_end]
    batch_int = np.tile(K[None, ...], (len(batch_paths), 1, 1))

    print(f"Processing batch {i} -> {batch_end}...")

    with torch.no_grad():
        predictions = model.inference(
            image=batch_paths,
            extrinsics=None,
            intrinsics=batch_int,
            align_to_input_ext_scale=False,
            export_dir=None,
            export_format="mini_npz"
        )

    depths = predictions.depth           # (B, H, W)
    pred_poses = predictions.extrinsics  # (B, 4, 4)

    for idx_in_batch, (depth_map, pose_matrix) in enumerate(zip(depths, pred_poses)):
        global_idx = i + idx_in_batch

        # DEPTH .NPY (Raw Float32)
        filename_npy = f"{global_idx:05d}_depth.npy"
        np.save(os.path.join(OUTPUT_DEPTH_DIR, filename_npy), depth_map)

        # PREDICTED POSE .TXT (W2C)
        filename_pose = f"{global_idx:05d}_pose.txt"
        np.savetxt(os.path.join(OUTPUT_DEPTH_DIR, filename_pose), pose_matrix)

        # Visualization
        filename_png = f"{global_idx:05d}.png"
        save_path = os.path.join(OUTPUT_DEPTH_DIR, filename_png)

        d_min, d_max = depth_map.min(), depth_map.max()
        if d_max - d_min > 0:
            d_norm = (depth_map - d_min) / (d_max - d_min) * 255.0
        else:
            d_norm = depth_map * 0
        d_color = cv2.applyColorMap(d_norm.astype(np.uint8), cv2.COLORMAP_INFERNO)
        cv2.imwrite(save_path, d_color)

    # Clean
    del predictions, depths, pred_poses
    torch.cuda.empty_cache()
    gc.collect()

print(f"Kết quả lưu tại: {OUTPUT_DEPTH_DIR}")

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



[WARN ] Dependency `gsplat` is required for rendering 3DGS. Install via: pip install git+https://github.com/nerfstudio-project/gsplat.git@0b4dddf04cb687367602c01196913cde6a743d70
🚀 KHỞI ĐỘNG BLIND POSE MODE - MODEL: da3-large
📦 Đang giải nén dataset...
📸 Tổng số ảnh: 1201
🧠 Đang load model da3-large...


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json: 0.00B [00:00, ?B/s]

[INFO ] using MLP layer as FFN


model.safetensors:   0%|          | 0.00/1.64G [00:00<?, ?B/s]

🏃 Bắt đầu chạy Batch (Model sẽ tự đoán Pose)...
    👉 Processing batch 0 -> 18...
[INFO ] Processed Images Done taking 0.162276029586792 seconds. Shape:  torch.Size([18, 3, 504, 504])
[INFO ] Selecting reference view using strategy: saddle_balanced
[INFO ] Model Forward Pass Done. Time: 3.89211368560791 seconds
[INFO ] Conversion to Prediction Done. Time: 0.02498006820678711 seconds
    👉 Processing batch 18 -> 36...
[INFO ] Processed Images Done taking 0.09360337257385254 seconds. Shape:  torch.Size([18, 3, 504, 504])
[INFO ] Selecting reference view using strategy: saddle_balanced
[INFO ] Model Forward Pass Done. Time: 2.0625617504119873 seconds
[INFO ] Conversion to Prediction Done. Time: 0.024381399154663086 seconds
    👉 Processing batch 36 -> 54...
[INFO ] Processed Images Done taking 0.08167052268981934 seconds. Shape:  torch.Size([18, 3, 504, 504])
[INFO ] Selecting reference view using strategy: saddle_balanced
[INFO ] Model Forward Pass Done. Time: 2.069908380508423 seconds
[

DA3MONO-LARGE

In [ ]:
import torch
import numpy as np
import os
import cv2
import zipfile
import gc
from pathlib import Path
from depth_anything_3.api import DepthAnything3

ZIP_PATH = "/content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip"
OUTPUT_DIR = "/content/drive/MyDrive/3D_reconstruction/simcol_output_mono"
OUTPUT_DEPTH_DIR = os.path.join(OUTPUT_DIR, "depth_data")

MODEL_NAME = "da3mono-large"
BATCH_SIZE = 18

# INTRINSIC
FX, FY = 227.60416, 237.5
CX, CY = 227.60416, 237.5

print(f"MODEL: {MODEL_NAME}")
os.makedirs(OUTPUT_DEPTH_DIR, exist_ok=True)

EXTRACT_PATH = "/content/dataset_simcol_rgb"
if not os.path.exists(EXTRACT_PATH):
    print(f"Extract dataset")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)

# List ảnh
image_extensions = {'.png', '.jpg'}
all_image_paths = sorted([
    str(p) for p in Path(EXTRACT_PATH).rglob("*")
    if p.suffix.lower() in image_extensions
])
total_imgs = len(all_image_paths)
print(f"Tổng số ảnh: {total_imgs}")

# Intrinsic
K = np.array([[FX, 0, CX], [0, FY, CY], [0, 0, 1]], dtype=np.float32)

# LOAD MODEL
print(f"🧠 Đang load model {MODEL_NAME}...")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DepthAnything3.from_pretrained(f"depth-anything/{MODEL_NAME}").to(device)

# RUNNING
print(f"Running Batch")

for i in range(0, total_imgs, BATCH_SIZE):
    batch_end = min(i + BATCH_SIZE, total_imgs)
    batch_paths = all_image_paths[i : batch_end]
    batch_int = np.tile(K[None, ...], (len(batch_paths), 1, 1))

    print(f"Processing batch {i} -> {batch_end}...")

    with torch.no_grad():
        predictions = model.inference(
            image=batch_paths,
            extrinsics=None,
            intrinsics=batch_int,
            align_to_input_ext_scale=False,
            export_dir=None,
            export_format="mini_npz"
        )

    depths = predictions.depth  # (B, H, W)

    for idx_in_batch, depth_map in enumerate(depths):
        global_idx = i + idx_in_batch
        filename = f"{global_idx:05d}"

        # DEPTH .NPY
        np.save(os.path.join(OUTPUT_DEPTH_DIR, f"{filename}.npy"), depth_map)

        # Visualization
        save_path_img = os.path.join(OUTPUT_DEPTH_DIR, f"{filename}.png")

        # Normalize
        d_min, d_max = depth_map.min(), depth_map.max()
        if d_max - d_min > 0:
            d_norm = (depth_map - d_min) / (d_max - d_min) * 255.0
        else:
            d_norm = depth_map * 0

        d_color = cv2.applyColorMap(d_norm.astype(np.uint8), cv2.COLORMAP_INFERNO)
        cv2.imwrite(save_path_img, d_color)

    # Clean
    del predictions, depths
    torch.cuda.empty_cache()
    gc.collect()

print(f"📂 Output tại: {OUTPUT_DEPTH_DIR}")

🚀 KHỞI ĐỘNG MONO MODE - MODEL: da3mono-large
📸 Tổng số ảnh: 1201
🧠 Đang load model da3mono-large...


config.json:   0%|          | 0.00/845 [00:00<?, ?B/s]

[INFO ] using MLP layer as FFN


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

🏃 Bắt đầu chạy Batch (Relative Depth)...
    👉 Processing batch 0 -> 18...
[INFO ] Processed Images Done taking 0.07575631141662598 seconds. Shape:  torch.Size([18, 3, 504, 504])
[INFO ] Model Forward Pass Done. Time: 1.050619125366211 seconds
[INFO ] Conversion to Prediction Done. Time: 0.009758949279785156 seconds
    👉 Processing batch 18 -> 36...
[INFO ] Processed Images Done taking 0.0797417163848877 seconds. Shape:  torch.Size([18, 3, 504, 504])
[INFO ] Model Forward Pass Done. Time: 0.995380163192749 seconds
[INFO ] Conversion to Prediction Done. Time: 0.009916543960571289 seconds
    👉 Processing batch 36 -> 54...
[INFO ] Processed Images Done taking 0.07736968994140625 seconds. Shape:  torch.Size([18, 3, 504, 504])
[INFO ] Model Forward Pass Done. Time: 0.9905836582183838 seconds
[INFO ] Conversion to Prediction Done. Time: 0.009920120239257812 seconds
    👉 Processing batch 54 -> 72...
[INFO ] Processed Images Done taking 0.07323169708251953 seconds. Shape:  torch.Size([18, 3